In [ ]:
from zero_shot_vocab_expansion import EmbeddingModel
from zero_shot_vocab_expansion.dataset import VocabDataset
from zero_shot_vocab_expansion.dataset import split_dataset
from zero_shot_vocab_expansion import EmbeddingModel
from sentence_transformers.losses import MSELoss
from torch.utils.data import DataLoader
import torch

print(f"GPU: {torch.cuda.is_available()}")


ds = VocabDataset("bert-base-uncased", select_definition="random")
train_ds, test_ds = split_dataset(ds, 0.8)
print(f"Training samples: {len(train_ds)}")
print(f"Testing samples: {len(test_ds)}")

In [ ]:
print(f"word: {ds[30].guid}, def: {ds[30].texts}")

All models, Datasets, DataLoaders, and training calls are based on the Sentence Transformers package by HuggingFace.  
https://www.sbert.net/index.html

In [ ]:
train_loader = DataLoader(train_ds, shuffle=True, batch_size=16)
# NOTE: next(iter(train_loader)) will not work. SentenceTransformers overrides default collate_fn
# Evaluator encodes all definitions for each word, then averages the definition embeddings before
# calculating MSE.
evaluator = test_ds.to_evaluator()
model = EmbeddingModel("bert-base-uncased")
loss = MSELoss(model)

In [ ]:
model.fit(train_objectives=[(train_loader, loss)], epochs=1, evaluator=evaluator)